# Hugging Face Transformers 시작하기

**학습 목표:**
- 현대 자연어 처리(NLP)의 판도를 바꾼 **트랜스포머(Transformer)** 아키텍처와 **어텐션(Attention)** 메커니즘의 개념을 이해합니다.
- **Hugging Face** 라이브러리의 강력한 `pipeline` API를 사용하여, 복잡한 코드 없이 사전 학습된(pre-trained) 모델을 활용하는 방법을 배웁니다.
- 감성 분석, 텍스트 생성, 마스크 채우기, 제로샷 분류 등 다양한 NLP 작업을 파이프라인으로 간단하게 수행해봅니다.
- 한국어 BERT 모델을 사용하여 한국어 텍스트 처리도 실습합니다.

In [ ]:
# transformers 라이브러리가 없다면 설치합니다.
!pip install transformers[sentencepiece]

In [ ]:
from transformers import pipeline
import pandas as pd

### (1) 파이프라인(Pipeline) API 이해하기
Hugging Face의 `pipeline`은 NLP 작업을 위한 가장 쉬운 인터페이스입니다. 원하는 작업의 이름을 문자열로 전달하면, 라이브러리가 다음 과정을 자동으로 처리해줍니다.
1.  해당 작업에 적합한 사전 학습된 모델을 허브에서 다운로드합니다.
2.  모델에 맞는 토크나이저(Tokenizer)를 다운로드하여 텍스트를 전처리합니다.
3.  처리된 입력을 모델에 전달하여 추론을 수행합니다.
4.  모델의 예측 결과를 사람이 이해하기 쉬운 형태로 후처리하여 반환합니다.

#### 작업 1: 감성 분석 (Sentiment Analysis)

In [ ]:
# 감성 분석 파이프라인 로드
classifier = pipeline('sentiment-analysis')

# 텍스트 리스트를 입력하여 한 번에 처리
texts = [
    "This movie is a masterpiece, a must-see for everyone!",
    "I felt this was a complete waste of my time and money.",
    "The acting was okay, but the plot was a bit boring."
]
results = classifier(texts)

for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f" -> Label: {result['label']}, Score: {result['score']:.4f}\n")

#### 작업 2: 텍스트 생성 (Text Generation)

In [ ]:
# 텍스트 생성 파이프라인 로드 (모델: gpt2)
generator = pipeline('text-generation', model='gpt2')

prompt = "In a world where artificial intelligence is king, humans"
generated_text = generator(prompt, max_length=60, num_return_sequences=1)

print(generated_text[0]['generated_text'])

#### 작업 3: 마스크 채우기 (Fill-Mask) - 한국어 모델 사용

In [ ]:
# 한국어 BERT 모델을 사용한 마스크 채우기 파이프라인 로드
unmasker = pipeline('fill-mask', model='klue/bert-base')

results_ko = unmasker("대한민국의 수도는 [MASK]이다.")

df = pd.DataFrame(results_ko)
print(df[['token_str', 'score']])

#### 작업 4: 제로샷 분류 (Zero-Shot Classification)
제로샷 분류는 모델을 특정 레이블에 대해 전혀 훈련시키지 않았음에도, 주어진 텍스트를 우리가 정의한 후보 레이블 중 하나로 분류하는 강력한 기능입니다.

In [ ]:
zero_shot_classifier = pipeline("zero-shot-classification", model='facebook/bart-large-mnli')

sequence_to_classify = "Who are you voting for in 2028?"
candidate_labels = ['politics', 'business', 'entertainment', 'sports']

result = zero_shot_classifier(sequence_to_classify, candidate_labels)
print(pd.DataFrame(result).to_string(index=False))